## 1. Importar Librerías

Usamos `AzureAIAgentClient` en lugar de `OpenAIChatClient` para trabajar directamente con Azure AI Foundry.

In [ ]:
import asyncio
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv
import os

## 2. Cargar Configuración

Necesitamos:
- `AZURE_PROJECT_ENDPOINT`: El endpoint del proyecto de Azure AI
- `MODEL`: El nombre del deployment del modelo

In [ ]:
load_dotenv()

project_endpoint = os.getenv("AZURE_PROJECT_ENDPOINT")
model = os.getenv("MODEL")

if not project_endpoint or not model:
    raise RuntimeError("Faltan AZURE_PROJECT_ENDPOINT y/o MODEL en .env")

print(f"✅ Endpoint: {project_endpoint}")
print(f"✅ Modelo: {model}")

## 3. Diferencia entre OpenAIChatClient y AzureAIAgentClient

### OpenAIChatClient:
- Usa endpoints de OpenAI (incluyendo Azure OpenAI)
- Requiere `base_url` + `/openai/v1/`
- Usa bearer token provider con DefaultAzureCredential

### AzureAIAgentClient:
- Específico para Azure AI Foundry
- Usa `project_endpoint` directamente
- Integración más profunda con Azure AI
- Usa AzureCliCredential para autenticación

## 4. Crear el Agente con Context Manager

El código usa `async with` para gestionar automáticamente los recursos. Esto es importante para:
- Cerrar conexiones correctamente
- Liberar recursos
- Manejo de errores limpio

In [ ]:
async def main():
    # Context manager para credential y agent
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                async_credential=credential, 
                project_endpoint=project_endpoint,
                model_deployment_name=model
            ),
            instructions="You are good at telling jokes."
        ) as agent,
    ):
        print("\n🤖 Agente creado con Azure AI Agent Client")
        print("\n💭 Preguntando al agente...\n")
        
        result = await agent.run("Tell me a joke about a pirate.")
        
        print("─" * 80)
        print("\n😄 Chiste del agente:\n")
        print(f"   {result.text}\n")
        print("─" * 80)

# Ejecutar
await main()

## 5. Autenticación con Azure CLI

`AzureCliCredential` usa las credenciales de Azure CLI. Esto significa que debes:

1. Tener Azure CLI instalado
2. Estar autenticado: `az login`
3. Tener acceso al recurso de Azure AI

**Ventaja**: No necesitas API keys en tu código o `.env`

## 6. Verificar Autenticación de Azure CLI

Ejecuta esto para verificar que estás autenticado:

In [ ]:
# Verificar si Azure CLI está configurado
import subprocess

try:
    result = subprocess.run(
        ["az", "account", "show"],
        capture_output=True,
        text=True,
        check=True
    )
    print("✅ Azure CLI está autenticado")
    print("\nInformación de la cuenta:")
    print(result.stdout)
except subprocess.CalledProcessError:
    print("❌ Azure CLI no está autenticado")
    print("\nEjecuta: az login")
except FileNotFoundError:
    print("❌ Azure CLI no está instalado")
    print("\nInstala desde: https://docs.microsoft.com/cli/azure/install-azure-cli")

## 7. Probar con Diferentes Prompts

In [ ]:
async def test_multiple_prompts():
    prompts = [
        "Tell me a joke about a programmer.",
        "Tell me a joke about AI.",
        "Tell me a joke about cloud computing."
    ]
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                async_credential=credential, 
                project_endpoint=project_endpoint,
                model_deployment_name=model
            ),
            instructions="You are good at telling jokes."
        ) as agent,
    ):
        for i, prompt in enumerate(prompts, 1):
            print(f"\n{'='*80}")
            print(f"CHISTE {i}")
            print("="*80)
            print(f"\n👤 Pregunta: {prompt}\n")
            
            result = await agent.run(prompt)
            
            print(f"🤖 Respuesta: {result.text}\n")

await test_multiple_prompts()

## Conclusión

Este ejemplo demuestra:

### 1. **Azure AI Agent Client**
- Integración nativa con Azure AI Foundry
- Autenticación mediante Azure CLI
- No requiere API keys en código

### 2. **Context Managers**
- Uso de `async with` para gestión de recursos
- Cierre automático de conexiones
- Código más limpio y seguro

### 3. **Comparación con OpenAIChatClient**

| Característica | OpenAIChatClient | AzureAIAgentClient |
|---------------|------------------|--------------------|
| Endpoint | `base_url + /openai/v1/` | `project_endpoint` |
| Autenticación | Bearer token + DefaultAzureCredential | AzureCliCredential |
| API Key | Opcional (con credential) | No necesaria |
| Plataforma | OpenAI / Azure OpenAI | Azure AI Foundry |
| Uso | General purpose | Azure AI específico |

### Cuándo usar cada uno:

**Usa OpenAIChatClient cuando:**
- Trabajas con Azure OpenAI Service
- Necesitas compatibilidad con OpenAI API
- Usas diferentes tipos de autenticación

**Usa AzureAIAgentClient cuando:**
- Trabajas con Azure AI Foundry
- Quieres integración más profunda con Azure AI
- Prefieres autenticación con Azure CLI
- Necesitas características específicas de Azure AI

### Ventajas de AzureCliCredential:
- ✅ Sin API keys en código
- ✅ Usa identidad de Azure
- ✅ Mejor seguridad
- ✅ Fácil rotación de credenciales
- ✅ Integración con RBAC de Azure

### Requisitos:
1. Azure CLI instalado
2. Autenticado: `az login`
3. Permisos en el recurso de Azure AI
4. Variables de entorno configuradas